# Spatial Operations in Geopandas

DATE: 11 June 2020, 18:00 - 21:00 UTC

AUDIENCE: Intermediate

INSTRUCTOR: Martin Bentley, Digital Geoscientist, [Agile](https://agilescientific.com/)

Importing and plotting vector data is all well and good, but we want to be able to do other things as well, like relating one dataset to another.

This notebook covers selection of features via some spatial relationships between different GeoDataFrames and selecting the data that we are interested in. There is also some code given for plotting rose diagrams (adapted from Bruno Ruas de Pinho's [post](http://geologyandpython.com/structural_geology.html)), which may be of interest.

In [ ]:
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
plt.rcParams["figure.figsize"] = (8, 8)

We are going to work with the runways located in South Africa, downloaded from OpenStreetMap. We will also load the outlines of each province from the Natural Earth dataset, and extract the South African ones. (Note: if you are actually working in South Africa, you should avoid these: they are very out of date now.)

In [ ]:
za_runways = gpd.read_file('../data/south_africa_runways.gpkg')
provinces = gpd.read_file('zip://../data/ne_10m_admin_1_states_provinces.zip')
za_provinces = provinces[provinces['adm0_a3'] == 'ZAF']

The `bearing_360` field in the runway dataset contains the direction each line that makes up a runway is pointing, which might be useful.

A rose diagram is quite a nice way to visualise this sort of data (anyone who has done a structural geology course will be familiar with these for various strutural readings, and they are commonly used for wind direction as well).

The easiest way to get a set of bins is by using the `histogram()` method from `numpy`. This returns two arrays: one with the value in each bin, and one with the edge of each bin. We will use 36 bins (so each is 10 degrees).

In [ ]:
bin_width_degrees = 10
bin_edges = np.arange(-5, 366, bin_width_degrees)
hist_counts, bin_edges = np.histogram(za_runways['bearing_360'], bin_edges)
print(f'Hist counts: {hist_counts}\nEdges: {bin_edges}')

print(list(hist_counts[:18]))
print(list(reversed(hist_counts[18:])))

We know that the first and last bins are the same (that is, bin 1 has edges at [-5, 5] and the last bin has edges [355, 365]), so we will make sure that we add those counts together. Then we can split the list in half before summing the counts from opposite bins.

In [ ]:
hist_counts[0] += hist_counts[-1]  # sum counts of first and last bins

half = np.sum(np.split(hist_counts[:-1], 2), 0)
two_halves = np.concatenate([half, half])
print(len(two_halves))
print(hist_counts)
print(two_halves)
print(list(two_halves[:18]))
print(list(two_halves[18:]))

These get used for the the parameters to the polar plot. `theta` is the centre of each bin, `radii` is the height of each bin, and `width` is how wide each bin needs to be.

In [ ]:
theta = np.linspace(0, 2 * np.pi, 36, endpoint=True)
radii = two_halves
width = np.radians(bin_width_degrees)

Finally, we can set up the plot. Note the addition of `polar` to the `projection` parameter. This is what tells matplotlib that we want a circular plot. By using `set_theta_offset` and `set_theta_direction` we can get 0 at the top and 90 to the right, as we expect from a compass. The grid can be altered using `set_theta_grids` and `set_rgrids` for the spokes and the rings respectively.

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='polar')
ax.bar(theta, radii, 
       width=width, bottom=0.0, edgecolor='k')
ax.set_theta_zero_location('N')
ax.set_theta_direction(-1)
ax.set_thetagrids(np.arange(0, 360, 10), labels=np.arange(0, 360, 10))
ax.set_rgrids(np.arange(0, radii.max() + 5, 5), angle=0)
ax.set_title('Directions of South African runways')

## Spatial Selections
This is useful, but it covers the entire country. If we want to look at each province in turn, because there may be differences, we can select only things in the province.

We will use the `within` function to see if a feature is within another. This returns a boolean (True/False) that we can use to select only the features for which this is true. There are a few of these worth noting:
* `contains` is the opposite of `within`, and is True for features that entirely contain a given geometry.
* `intersects` is true if the interior and interior of the two objects overlap.
* `touches` is true if at least one point is in common and interiors do not intersect.

In addition to these, most other spatial operations that are present in standard GIS software are available.

First, we will select the province of KwaZulu-Natal and the runways within it, and then plot them.

In [ ]:
kzn = za_provinces[za_provinces['iso_3166_2'] == 'ZA-NL']
kzn_runways = za_runways[za_runways.within(kzn.geometry.unary_union)]

In [ ]:
print(f'All runways: {za_runways.shape}\nKZN runways: {kzn_runways.shape}')

In [ ]:
base = za_provinces.plot(color='lightgrey', edgecolor='darkgrey')
kzn_runways.centroid.plot(ax=base, color='black', alpha=0.3)
base.set_ylim(-35.2, -21.8)  # These limits are to ignore the Gough Islands
base.set_xlim(16, 33.2)  # These limits are to ignore the Gough Islands

We can see that this has only extracted values within the province's boundaries. We needed the `unary_union` because the geometry is that of a multipolygon.

## Exercise 1

1. Convert the above code to a function for creating a rose diagram given a geoDataFrame
    1. Test it on the extracted runway directions in KZN.
2. Write a function that generates a rose diagram for each province. The easiest approach is to wrap the previous function. (Hint: you may find the `iterrows()` method that (geo)DataFrames have useful.)

In [ ]:
# Write a function to plot a rose diagram.
def rose_diagram(bearings, title='Rose Diagram', degrees_in_bin=10):
    bin_width_degrees = degrees_in_bin
    num_bins = int(round(360 / degrees_in_bin, 0))
    
    bin_edges = np.arange(-5, 366, bin_width_degrees)
    hist_counts, bin_edges = np.histogram(bearings, bin_edges)
    
    hist_counts[0] += hist_counts[-1]  # sum counts of first and last bins

    half = np.sum(np.split(hist_counts[:-1], 2), 0)
    two_halves = np.concatenate([half, half])
    
    theta = np.linspace(0, 2 * np.pi, num_bins, endpoint=True)
    radii = two_halves
    width = np.radians(bin_width_degrees)
    
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='polar')
    ax.bar(theta, radii, 
           width=width, bottom=0.0, edgecolor='k')
    ax.set_theta_zero_location('N')
    ax.set_theta_direction(-1)
    ax.set_thetagrids(np.arange(0, 360, 10), labels=np.arange(0, 360, 10))
    ax.set_rgrids(np.arange(0, radii.max() + 5, 5), angle=0)
    ax.set_title(title)

In [ ]:
# Test the function on the runways in KwaZulu-Natal.


In [ ]:
# Test the function on the runways in KwaZulu-Natal.
rose_diagram(kzn_runways['bearing'], 'Direction of runways in KZN')

In [ ]:
# Plot a rose diagram for each province.




In [ ]:
# Plot a rose diagram for each province.
for index, province in za_provinces.iterrows():
    province_runways = za_runways[za_runways.within(province.geometry)]
    rose_diagram(province_runways['bearing_360'], f'Direction of runways in {province["name"]}')

## Spatial Relationships

Now we will take a look at a few places in KZN, and get an idea of some spatial relationships between towns/cities and runways.

In [ ]:
za_places = gpd.read_file("../data/south_africa_places.gpkg")

We can now look at how many airports are located within a certain range of a given town. First we will choose our town, then create a buffer around, and see how many fall into that range. I am going to look at Durban, the largest city in KZN.

In [ ]:
durban = za_places[za_places['name'] == 'Durban']
print(durban.geometry)
durban

Geopandas gives us some useful and common tools straight away, such as a buffer. We can easily put a circle around a given geometry.

In [ ]:
base = kzn.plot(color='lightgrey')
durban.buffer(1).plot(ax=base, alpha=0.5)
durban.plot(ax=base, zorder=4, color='red')

That looks a little big, given that our buffer was only `1`. If we look at the CRS of the place data, we can see that it works in degrees, so the buffer is one degree in each direction.We should convert this to a projected system that uses metres if we want to measure meaningful distances. We will try EPSG 3857, which uses metres as a unit of measure. (For our purposes, it is accurate enough, but if you need distances in a critical use-case, then you should use something more focused on your area of interest.)

For smaller areas, a relevant UTM projection is good. Some countries also have well-defined ones of their own. https://gist.github.com/gubuntu/6403425 is of interest for Southern Africa generally. KZN is a little too big to fit into one UTM zone.

In [ ]:
kzn_runways_proj = kzn_runways.to_crs(epsg=3857)
kzn_proj = kzn.to_crs(epsg=3857)
durban_proj = durban.to_crs(epsg=3857)

Notice how this also changes our axes when plotting:
(We are plotting the centroid of each runway line here, to make the position more visible.)

In [ ]:
base = kzn_proj.plot(color='lightgrey')
durban_proj.buffer(100_000).plot(ax=base, alpha=0.5)  # 100 000m is 100km
durban_proj.plot(ax=base, zorder=4, color='red')
kzn_runways_proj.centroid.plot(ax=base)

Now we can use that buffer and select only the runways within that buffer.

In [ ]:
durban_proj_buffer = durban_proj.buffer(100_000)
near_durban = kzn_runways_proj[kzn_runways_proj.within(durban_proj_buffer.unary_union)]

We can now plot only those runways that fall within the buffer.

In [ ]:
fig, ax = plt.subplots()
kzn_proj.plot(ax=ax, color='lightgrey')
near_durban.centroid.plot(ax=ax)
durban_proj_buffer.plot(ax=ax, alpha=0.3)
durban_proj.plot(ax=ax, color='red')

## Distances and Nearest Neighbours

If we are interested in the distance to the nearest runway from one of our towns, we can write a short function that returns this value. A GeoDataFrame has a `distance` method that will work between any two geometries. Since we want to do it for a number of them in one go, we have to write a function to do it.

In [ ]:
def min_distance(point, destination):
    '''
    Takes a point and then returns the nearest destination. 
    Expects to be working with metres as a unit of measure and returns a distance in km.
    '''
    return destination.distance(point).min() / 1000

The paradigm in pandas is to use `apply` which performs the same function on each row in a dataframe.

Since we know that the nearest runway will be one of the ones that we previously identified as being within the buffer zone, we will just use that. We could give it the whole dataset and get the same result, but this will be slightly quicker.

The final comma in `args=(near_durban,)` is important.

In [ ]:
durban_proj.geometry.apply(min_distance, args=(near_durban,))

## Exercise 3

1. What is the distance to the centre of the closest town to the runway with the `osm_id` of 87323625?
2. What is the distance from Durban to the furtherest town in South Africa? (Hint: you will need to modify the `nearest_neighbour` function to a `max_distance` version. Pay attention to the CRS.)

In [ ]:
# What is the distance to the centre of the closest town to the runway with the osm_id of 87323625?



In [ ]:
# What is the distance to the centre of the closest town to the runway with the osm_id of 87323625?
runway87323625 = kzn_runways_proj[kzn_runways_proj['osm_id'] == '6211489']
runway87323625.geometry.apply(min_distance, args=(za_places.to_crs(epsg=3857),))

In [ ]:
# What is the distance from Durban to the furtherest town in South Africa?





In [ ]:
# What is the distance from Durban to the furtherest town in South Africa?
def max_distance(point, destination):
    return destination.distance(point).max() / 1000

durban_proj.geometry.apply(max_distance, args=(za_places.to_crs(epsg=3857),))

## Multiple points

We can also use the `nearest_neighbours` function applied to every feature. This can be used for example, to classify the runways by what the closest town or city to them is.

First we will limit our analysis to towns in KwaZulu-Natal.

In [ ]:
kzn_places = za_places[za_places.within(kzn.geometry.unary_union)]
kzn_places_proj = kzn_places.to_crs(epsg=3857)

In [ ]:
base = kzn_proj.plot(color='lightgrey')
kzn_runways_proj.centroid.plot(ax=base)
kzn_places_proj.plot(ax=base, color='red', marker='+')

Using the `apply` function on the whole GeoDataFrame will get us a result for all the features. We can use that to populate a new column in the runway dataset with the minimum distance to a town or city in the places dataset.

In [ ]:
kzn_runways_proj['min_dist_to_place'] = kzn_runways_proj.geometry.apply(min_distance, args=(kzn_places_proj,))
kzn_runways_proj['min_dist_to_place'].describe()

The runways end up plotting very small at province-wide scale, so we will change the geometry to be a Point instead. The easiest way is to obtain the centroid of each runway.

In [ ]:
kzn_runways_proj['line_geom'] = kzn_runways_proj.geometry
kzn_runways_proj['geometry'] = kzn_runways_proj.centroid
kzn_runways_proj.geometry

Using the `scheme` parameter, we can also take a column and group the values using `mapclassify` options.

In [ ]:
base = kzn_proj.plot(color='lightgrey')
kzn_runways_proj.plot(ax=base, column='min_dist_to_place', legend=True, scheme='EqualInterval', k=7)
kzn_places_proj.plot(ax=base, color='red', marker='+')

## Exercise

1. Plot the towns and cities in KwaZulu-Natal coloured by the distance from Durban.

In [ ]:
kzn_durban = kzn_places_proj.copy()
kzn_durban['dist_durban'] = kzn_places_proj.geometry.apply(min_distance, args=(durban_proj,))
base = kzn_proj.plot(color='lightblue', edgecolor='black')
kzn_durban.plot(ax=base, column='dist_durban', scheme='jenkscaspallforced', k=6, legend=True, cmap='Reds_r', marker='+')
durban_proj.plot(ax=base, color='red')

## More Information from Neighbours

So far we have only been able to obtain the distance to the nearest neighbour, but nothing else from the second GeoDataFrame.

We need a new function that can get the nearest points to a given geometry. To do that we will use an operation imported from the `shapely` module.

In [ ]:
from shapely.ops import nearest_points

def calculate_nearest(row, destination, val, col="geometry"):
    ''' Returns the value from a given column for the nearest feature
        of the `destination` GeoDataFrame.
        
        row: The object of interest
        destination: GeoDataFrame of possible locations, of which one is the closest.
        val: The column containing the values which the destination will give back.
        col: If your row's geometry is not `geometry`, change it here.
        
        returns: The value of the `val` column of the feature nearest to `row`.
    '''
    dest_unary = destination["geometry"].unary_union
    nearest_geom = nearest_points(row[col], dest_unary)
    match_geom = destination.loc[destination.geometry == nearest_geom[1]]
    match_value = match_geom[val].to_numpy()[0]
    return match_value

This function allows us to ask for specific values back about the nearest object. Again, we will find the nearest place to each runway, but we will request the `name` of that place and save it as a column in the runways GeoDataFrame.

In [ ]:
kzn_runways_proj["nearest_place_name"] = kzn_runways_proj.apply(
    calculate_nearest, destination=kzn_places_proj, val="name", axis=1)

In [ ]:
kzn_runways_proj["nearest_place_name"].head(10)

We can now plot these and get an idea of where the nearest urban centre to each runway is.

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))
kzn_proj.plot(ax=ax, zorder=-4, color='lightgrey')
kzn_places_proj.plot(ax=ax, color='black', marker='+',)
kzn_runways_proj.plot(ax=ax, column='nearest_place_name', legend=True, cmap='tab20b')
ax.set_title('Nearest major place to runways in KZN')
ax.set_xlim(xmin=kzn_proj.total_bounds[0] - 10_000, xmax=kzn_proj.total_bounds[2] + 135_000)

## Exercise 3

1. Is there a spatial trend in the bearing of runways in KZN? Plot the position of each runway and colour them by bearing.
2. Plot the runways within 150km of Pietermaritzburg and colour them by runway length. This plot should only cover 150km of Pietermaritzburg.

In [ ]:
# Is there a spatial trend in the bearing of runways in KZN?
# Plot the posiiton of each runway and colour them by bearing.





In [ ]:
# Is there a spatial trend in the bearing of runways in KZN?
# Plot the posiiton of each runway and colour them by bearing.
fig, ax = plt.subplots(figsize=(10,10))
kzn_proj.plot(ax=ax, zorder=-4, color='lightgrey')
kzn_runways_proj.plot(ax=ax, column='bearing', legend=True, scheme='EqualInterval')
#kzn_places_proj.plot(ax=ax, color='black')
ax.set_title('Bearing of runways in KZN')

In [ ]:
# Plot the runways within 150km of Pietermaritzburg and colour them by runway length.
# This plot should only cover 150km of Pietermaritzburg.



In [ ]:
# Plot the runways within 150km of Pietermaritzburg and colour them by runway length.
# This plot should only cover 150km of Pietermaritzburg.
pmb_buffer = kzn_places_proj[kzn_places_proj['name'] == 'Pietermaritzburg'].buffer(150_000)
kzn_runways_proj[kzn_runways_proj.within(pmb_buffer.geometry.unary_union)].plot(column='length')

<hr />
<img src="https://avatars1.githubusercontent.com/u/1692321?v=3&s=200" style="float:center" width="40px" />
<p><center>© 2020 <a href="http://www.agilegeoscience.com/">Agile Geoscience</a> — <a href="https://creativecommons.org/licenses/by/4.0/">CC-BY</a></center></p>